# 下载Google earth data

In [3]:
import ee
import geemap
import os
import math
import geopandas as gpd
# 用户验证
# ee.Authenticate()
# 初始化Google Earth Engine
ee.Initialize()

In [4]:
# 切分区域
def split_region(region, col_size, row_size):
    bounds = region.getInfo()['coordinates'][0]
    original_minx, original_miny = bounds[0]
    original_maxx, original_maxy = bounds[2]
    
    x_len = original_maxx - original_minx
    y_len = original_maxy - original_miny
    
    # 向上取整
    cols = math.ceil(x_len / col_size)
    rows = math.ceil(y_len / row_size)
    
    rectangles = []
    for i in range(cols):
        current_minx = original_minx + i * col_size
        current_maxx = current_minx + col_size
        if current_maxx > original_maxx:
            current_maxx = original_maxx
        for j in range(rows):
            current_miny = original_miny + j * row_size
            current_maxy = current_miny + row_size
            if current_maxy > original_maxy:
                current_maxy = original_maxy
            rect = ee.Geometry.Rectangle([current_minx, current_miny, current_maxx, current_maxy])
            rectangles.append(rect)
    return rectangles

In [5]:
# 读取本地Shapefile 得到处理区域
shapefile_path = r"F:\ArcgisData\shp_polygon\KY500.shp"
gdf = gpd.read_file(shapefile_path)


bounds = gdf['geometry'][0].bounds
bbox = {
    'MinX': bounds[0],
    'MinY': bounds[1],
    'MaxX': bounds[2],
    'MaxY': bounds[3]
}

# 获取第一个几何对象
geometry = gdf.geometry.iloc[0]
# 2. 将 Earth Engine 几何对象转换为 Feature 对象
ee_geometry = ee.Geometry(geometry.__geo_interface__)
# 3. 将 Earth Engine 几何对象转换为 Feature 对象
feature = ee.Feature(ee_geometry)

# 4. 将 Feature 对象放入 FeatureCollection 中
feature_collection = ee.FeatureCollection([feature])

# 获取 FeatureCollection 的边界几何对象
bounded_geometry = feature_collection.geometry().bounds()


In [6]:
# 筛选无云区域QA60
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)
# 筛选无云区域SCL
def mask_s2_clouds_scl(image):
    """Masks clouds in a Sentinel-2 image using the SCL band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    scl = image.select('SCL')

    # Define the cloud mask. Typically, values 3, 8, 9, 10, and 11 in the SCL
    # band represent cloud and shadow conditions.
    cloud_shadow = scl.eq(3)  # Cloud shadows
    cloud_medium_prob = scl.eq(8)  # Medium probability clouds
    cloud_high_prob = scl.eq(9)  # High probability clouds
    thin_cirrus = scl.eq(10)  # Thin cirrus
    cloud_mask = cloud_shadow.Or(cloud_medium_prob).Or(cloud_high_prob).Or(thin_cirrus)

    return image.updateMask(cloud_mask.Not()).divide(10000)


# 定义要下载的图像的参数
area_of_interest =ee.Geometry.Rectangle([bbox['MinX'], bbox['MinY'], bbox['MaxX'], bbox['MaxY']])  # 请替换经度和纬度
start_date = '2022-12-01'  # 开始日期
end_date = '2023-12-30'    # 结束日期
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12','QA60','SCL']  # 选择要下载的波段

# 创建Sentinel-2图像集合
dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start_date, end_date) \
    .filterBounds(area_of_interest) \
    .select(bands) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(mask_s2_clouds_scl)

In [7]:
polygon_list = split_region(bounded_geometry,0.05,0.05)

In [8]:
len(polygon_list)

132

In [9]:
import os
# 设置输出文件路径
output_folder = r"F:\GEEDOWNLOAD\sentinel2\KY"  # 输出文件夹路径
output_file_prefix = 'Sentinel-2_scl'
idx=1
for one_polygon in polygon_list:
    file_name = os.path.join(output_folder,f"{output_file_prefix}_{idx}.tif")
    if os.path.exists(file_name):
        print('已存在')
    else:
        geemap.ee_export_image(dataset.median(), filename=file_name, region=one_polygon,scale=10)
    idx+=1
    print(idx)

Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_1.tif
2
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_2.tif
3
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_3.tif
4
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_4.tif
5
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_5.tif
6
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_6.tif
7
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_7.tif
8
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_8.tif
9
Generating URL ...
Please wait ...
Data downloaded to F:\GEEDOWNLOAD\sentinel2\KY\Sentinel-2_scl_9.tif
10
Generating URL ...
Please wait ...
Data downloaded to 

In [19]:
# 在地图中查看筛选结果
# def mask_s2_clouds(image):
#   """Masks clouds in a Sentinel-2 image using the QA band.

#   Args:
#       image (ee.Image): A Sentinel-2 image.

#   Returns:
#       ee.Image: A cloud-masked Sentinel-2 image.
#   """
#   qa = image.select('QA60')

#   # Bits 10 and 11 are clouds and cirrus, respectively.
#   cloud_bit_mask = 1 << 10
#   cirrus_bit_mask = 1 << 11

#   # Both flags should be set to zero, indicating clear conditions.
#   mask = (
#       qa.bitwiseAnd(cloud_bit_mask)
#       .eq(0)
#       .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
#   )

#   return image.updateMask(mask).divide(10000)
def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the SCL band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    scl = image.select('SCL')

    # Define the cloud mask. Typically, values 3, 8, 9, 10, and 11 in the SCL
    # band represent cloud and shadow conditions.
    cloud_shadow = scl.eq(3)  # Cloud shadows
    cloud_medium_prob = scl.eq(8)  # Medium probability clouds
    cloud_high_prob = scl.eq(9)  # High probability clouds
    thin_cirrus = scl.eq(10)  # Thin cirrus
    cloud_mask = cloud_shadow.Or(cloud_medium_prob).Or(cloud_high_prob).Or(thin_cirrus)

    return image.updateMask(cloud_mask.Not()).divide(10000)

dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2022-10-01', '2023-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.addLayerControl()
m.set_center(107.45, 26.12, 12)
m.add_layer(dataset.median(), visualization, 'RGB')
m

Map(center=[26.12, 107.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…